# Custom User Model

Django’s built-in User model allows us to start working with users right away, as we just did with our Blog app in the previous chapters. However, most
large projects need a way to add information related to users, such as age or any number of additional fields. There are two popular approaches.

The first-and older approach-is called the “User Profile” approach and extends the existing User model by creating a OneToOneField to a separate model
containing fields with additional information. The idea is to keep authentication reserved for User and not bundled with non-authentication-related user
information.

The second approach, to create a custom user model, is recommended in the official Django documentation. We can extend AbstractUser to create a
custom user model that behaves identically to the default User model but provides the option for customization in the future.


We will use AbstractUser to create a custom user model for a new Newspaper website project. The choice of a newspaper app pays homage to
Django’s roots as a web framework built for editors and journalists at the Lawrence Journal-World.

### Initial Set Up

The first step is to create a new Django project from the command line. We need to do our familiar steps of creating and navigating to a new directory called
news and installing and activating a new virtual environment called . venv.

```shell
$ mkdir news
$ cd news
$ python3 -m venv .venv
$ source .venv/bin/activate
(.venv) $
```

Next, install Django and Black, create a new Django project called django_project, and make a new app called accounts.
Shell

```shell
(.venv) $ python -m pip install django~=4.2.0
(.venv) $ python -m pip install black
(.venv) $ django-admin startproject django_project .
(.venv) $ python manage.py startapp accounts
```

Note that we did not run migrate to configure our database. It’s important to wait until after we've created our new custom user model before doing so,
given how tightly connected the user model is to the rest of Django.
In your web browser, navigate to http://127.0.0.1:8000, and the familiar Django welcome screen will be visible.

### Git
The start of a new project is an excellent time to initialize Git and create a repo on GitHub. We've done this several times before so we can use the same
commands to initialize a new local Git repo and check its status.
Shell

```shell
(.venv) $ git init
(.venv) $ git status
```

The .venv directory and the SQLite database should not be included in Git, so create a project-level .gitignore file in your text editor. 
We can also add the __pycache__ directory while at it.

.gitignore
.venv/
__pycache__/
*.sqlite3

Run git status again to confirm the .venv directory and SQLite database are not included. Then add the rest of our work along with a commit message.

```bash
(.venv) $ git status
(.venv) $ git add -A
(.venv) $ git commit -m "initial commit"
```

Create a new repo on GitHub and provide a name. Make sure to use your repo name and username
with the command below.
Shell

(.venv) § git remote add origin https://github.com/...........
(.venv) § git branch -M main
(.venv) $ git push -u origin main


All done!

### Custom User Model

Creating our custom user model requires four steps:

- updatedjango_project/settings.py
- create a new CustomUser model
- create new forms for UserCreationFormand UserChangeForm
- update accounts/admin.py

In django_project/settings.py, we'll add the accounts app to our INSTALLED_APPS. Then at the bottom of the file, use the AUTH_USER_MODEL
config to tell Django to use our new custom user model instead of the built-in User model. We'll call our custom user model CustomUser. Since it will
exist within our accounts app, we should refer to it as accounts. CustomUser.
Code

```python
# django_project/settings.py
INSTALLED_APPS = [

"django.contrib.admin",

"django.contrib.auth",

"django.contrib.contenttypes",

"django.contrib.sessions",

"django.contrib.messages",

"django.contrib.staticfiles",

"accounts", # new
]
AUTH_USER_MODEL = "accounts.CustomUser" # new
```

Now update accounts/models.py with a new User model called CustomUser that extends the existing AbstractUser. We also include a custom
field for age here.

Code

```python
# accounts/models.py
from django.contrib.auth.models import AbstractUser
from django.db import models

class CustomUser (AbstractUser):
age = models.PositiveIntegerField(null=True, blank=True)
```



If you read the documentation on custom user models, it recommends using AbstractBaseUser, not AbstractUser, which complicates things for
beginners. Working with Django is far simpler and remains customizable if we use AbstractUser instead.
So why use AbstractBaseUser at all? If you want a fine level of control and customization, AbstractBaseUser can be justified. But it requires
rewriting a core part of Django. If we want a custom user model that can be updated with additional fields, the better choice is AbstractUser, which
subclasses AbstractBaseUser. In other words, we write much less code and have less opportunity to mess things up. It’s the better choice unless you
really know what you're doing with Django!
Note that we use both null and blank with our age field. These two terms are easy to confuse but quite distinct:

- null is database-related. When a field has nul1=True, it can store a database entry as NULL, meaning no value.

- blank is validation-related. If blank=True, then a form will allow an empty value, whereas if blank=False then a value is required.

In practice, null and blank are commonly used together in this fashion so that a form allows an empty value, and the database stores that value as NULL.
A common gotcha to be aware of is that the field type dictates how to use these values. Whenever you have a string-based field like CharField or
TextField, setting both null and blank as we've done will result in two possible values for “no data” in the database, which is a bad idea. Instead, the
Django convention is to use the empty string " ", not NULL.